In [1]:
import torch
import pandas as pd

# 修改为你实际的模型路径
model_path_1 = "/data/home/yekai/github/DiffAdvPerturbationBench/multi-dreambooth/train_outputs/SimAC_SD21_VGGFace2_10ids_r8_cleanordirty/2000/unet/diffusion_pytorch_model.bin"
model_path_2 = "/data/home/yekai/github/DiffAdvPerturbationBench/SD/stable-diffusion-2-1-base/unet/diffusion_pytorch_model.bin"

# 加载权重
sd1 = torch.load(model_path_1, map_location="cpu")
sd2 = torch.load(model_path_2, map_location="cpu")

# 所有键
all_keys = sorted(set(sd1.keys()) | set(sd2.keys()))

results = []
for key in all_keys:
    if key not in sd1:
        results.append((key, "Missing in Model 1"))
    elif key not in sd2:
        results.append((key, "Missing in Model 2"))
    else:
        # 比较张量是否完全一致
        same = torch.equal(sd1[key], sd2[key])
        results.append((key, "Same" if same else "Different"))

# 输出为表格
df = pd.DataFrame(results, columns=["Parameter Name", "Status"])
print(df)

/tmp/ipykernel_704516/2712004162.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd1 = torch.load(model_path_1, map_location="cpu")
/tmp/ipykernel_704516/2712004162.py:1

                                 Parameter Name     Status
0                                  conv_in.bias       Same
1                                conv_in.weight       Same
2                            conv_norm_out.bias       Same
3                          conv_norm_out.weight       Same
4                                 conv_out.bias       Same
..                                          ...        ...
681          up_blocks.3.resnets.2.norm1.weight  Different
682            up_blocks.3.resnets.2.norm2.bias  Different
683          up_blocks.3.resnets.2.norm2.weight  Different
684    up_blocks.3.resnets.2.time_emb_proj.bias  Different
685  up_blocks.3.resnets.2.time_emb_proj.weight  Different

[686 rows x 2 columns]


In [2]:
df.to_csv("unet_diff_report.csv", index=False)